In [1]:
from corpus_characterizer import generator_chunker
import numpy as np
import os
# from sklearn.metrics import mean_squared_error,mean_absolute_error, median_absolute_error, mean_squared_log_error, explained_variance_score, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, explained_variance_score, \
    r2_score
import pandas as pd
from scipy.stats import describe, kurtosistest, skewtest, normaltest
from Conv1D_LSTM_Ensemble import pair_generator_1dconv_lstm_bagged
from Conv1D_ActivationSearch_BigLoop import pair_generator_1dconv_lstm #NOT BAGGED
from AuxRegressor import create_testing_set, create_training_set
import matplotlib.pyplot as plt 

# @@@@@@@@@@@@@@ RELATIVE PATHS @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
Base_Path = "./"
image_path = "./images/"
train_path = "./train/"
test_path = "./test/"
analysis_path = "./analysis/"
chunker_path = analysis_path + "chunker/"
preds_path = analysis_path + "preds/"

#create the data-pair filenames (using zip), use the helper methods
train_set_filenames = create_training_set()
test_set_filenames = create_testing_set()
print(train_set_filenames)
print(test_set_filenames)

Using Theano backend.


after shuffling: [('sequence_3a_709_11_fv1c.npy', 'sequence_3a_709_11_label_fv1c.npy'), ('sequence_3a_613_15_fv1c.npy', 'sequence_3a_613_15_label_fv1c.npy'), ('sequence_1d_225_6_fv1c.npy', 'sequence_1d_225_6_label_fv1c.npy'), ('sequence_1d_126_7_fv1c.npy', 'sequence_1d_126_7_label_fv1c.npy'), ('sequence_3a_577_18_fv1c.npy', 'sequence_3a_577_18_label_fv1c.npy'), ('sequence_1d_124_5_fv1c.npy', 'sequence_1d_124_5_label_fv1c.npy'), ('sequence_1d_251_2_fv1c.npy', 'sequence_1d_251_2_label_fv1c.npy'), ('sequence_1b_55_6_fv1c.npy', 'sequence_1b_55_6_label_fv1c.npy'), ('sequence_1l_384_5_fv1c.npy', 'sequence_1l_384_5_label_fv1c.npy'), ('sequence_1c_72_3_fv1c.npy', 'sequence_1c_72_3_label_fv1c.npy'), ('sequence_1c_102_3_fv1c.npy', 'sequence_1c_102_3_label_fv1c.npy'), ('sequence_1c_87_8_fv1c.npy', 'sequence_1c_87_8_label_fv1c.npy'), ('sequence_1d_174_15_fv1c.npy', 'sequence_1d_174_15_label_fv1c.npy'), ('sequence_2c_541_2_fv1c.npy', 'sequence_2c_541_2_label_fv1c.npy'), ('sequence_1d_252_3_fv1c.npy

In [2]:
#features_list = ['percent_damage','delta_K_current_1','ctip_posn_curr_1','delta_K_current_2','ctip_posn_curr_2',
          #'delta_K_current_3','ctip_posn_curr_3','delta_K_current_4','ctip_posn_curr_4','Load_1','Load_2']
    
# train_list = ['StepIndex','delta_K_current_1','ctip_posn_curr_1','delta_K_current_2','ctip_posn_curr_2',
#           'delta_K_current_3','ctip_posn_curr_3','delta_K_current_4','ctip_posn_curr_4','Load_1','Load_2'] #and seq_id,somehow 
unique_init_conds = []
training_seqname_initcond_dict = {}
combined_seqname_initcond_dict = {}

for files in train_set_filenames:
    data_load_path = train_path + '/data/' + files[0]
    label_load_path = train_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    train_array = np.load(data_load_path)
    train_label_array = np.load(label_load_path)[:, 1:]
    if train_array.shape[1] == 12:
        train_array = train_array[:,1:] #skip step_index. 
    if "fv1c" not in str(files[0]):
        print(train_array[0,:])
        #init_cond = train_array[0,[3,5,7,9]]
        init_cond = train_array[0,[2,4,6,8]]
    if "fv1c" in str(files[0]):
        init_cond = train_array[0,[2,4,6,8]]
    #print(str(files[0])," init cond",init_cond)
    print(str(files[0])," init cond2",train_array[0,:])
    print(str(files[0])," init cond2",train_array[1,:])
    training_seqname_initcond_dict[str(files[0])] = str(init_cond)
    
    array_equality = False
    if len(unique_init_conds) == 0:
            unique_init_conds.append(init_cond)
    if len(unique_init_conds) != 0:
        for ic in unique_init_conds:
            if np.array_equal(init_cond,ic) == True:
                #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                array_equality = True
    if array_equality == False:
        unique_init_conds.append(init_cond) #the load types
    #print("first 6 rows:",train_array[0:2,:])
print("there are ",len(unique_init_conds)," init conds in the training set")
print("unique init conds:",unique_init_conds)

('sequence_3a_709_11_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.610625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.28    ]))
('sequence_3a_709_11_fv1c.npy', ' init cond2', array([ 27.00025342,   8.24998091,   0.61062629,   8.69953772,
         1.33562346,   8.52377605,   1.51562638,   8.21240627,
         2.13062373,  14.28      ,   0.5       ]))
('sequence_3a_613_15_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.610625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.28    ]))
('sequence_3a_613_15_fv1c.npy', ' init cond2', array([ 27.00028525,   8.47764173,   0.61062643,   8.47764173,
         1.33562357,   8.47764173,   1.51562643,   8.47764173,
         2.13062357,  14.28      ,   1.75      ]))
('sequence_1d_225_6_fv1c.npy', ' init cond2', array([ 27.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  1

('sequence_1b_23_4_fv1c.npy', ' init cond2', array([  5.      ,   0.      ,   0.490625,   0.      ,   1.365625,
         0.      ,   1.415625,   0.      ,   2.253125,   1.428   ,  14.28    ]))
('sequence_1b_23_4_fv1c.npy', ' init cond2', array([  5.00029459,   8.13514462,   0.49062367,   8.78664099,
         1.36562353,   8.41813685,   1.41562633,   8.72110114,
         2.25312344,  14.28      ,   1.75      ]))
('sequence_1d_210_21_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.410625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.28    ]))
('sequence_1d_210_21_fv1c.npy', ' init cond2', array([ 27.00027268,   7.80114941,   0.41062392,   9.02468255,
         1.33562325,   8.89851245,   1.51562667,   8.36142836,
         2.13062364,  14.28      ,   0.75      ]))
('sequence_3a_605_7_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.620625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625

('sequence_1b_31_2_fv1c.npy', ' init cond2', array([  5.      ,   0.      ,   0.490625,   0.      ,   1.365625,
         0.      ,   1.415625,   0.      ,   2.253125,   1.428   ,  14.28    ]))
('sequence_1b_31_2_fv1c.npy', ' init cond2', array([  5.00035983,   8.53453734,   0.49062357,   9.29497954,
         1.36562325,   9.11538286,   1.4156268 ,   8.75225698,
         2.25312355,  14.28      ,   0.75      ]))
('sequence_1l_421_2_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.410625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.28    ]))
('sequence_1l_421_2_fv1c.npy', ' init cond2', array([ 27.00029748,   8.997125  ,   0.41062328,   7.34078772,
         1.33562411,   8.10831476,   1.51562626,   8.5903845 ,
         2.13062351,  14.28      ,   1.428     ]))
('sequence_3a_711_13_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.610625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,

         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  12.28    ]))
('sequence_3a_659_1_fv1c.npy', ' init cond2', array([ 27.00012797,   7.04889581,   0.62062579,   7.21392305,
         1.33562414,   7.51171437,   1.51562606,   6.63838074,
         2.13062436,  12.28      ,   1.428     ]))
('sequence_3a_670_12_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.610625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  12.28    ]))
('sequence_3a_670_12_fv1c.npy', ' init cond2', array([ 27.00011169,   6.06997619,   0.6106255 ,   7.51466249,
         1.335624  ,   6.92432082,   1.51562576,   6.29534942,
         2.13062444,  12.28      ,   1.428     ]))
('sequence_1k_307_8_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.410625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.28    ]))
('sequence_1k_307_8_fv1c.npy', ' init cond2', array([ 27.00043188,   8.14895397,   

('sequence_3a_620_2_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.620625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.28    ]))
('sequence_3a_620_2_fv1c.npy', ' init cond2', array([ 27.00030354,   7.94421314,   0.62062624,   9.04223944,
         1.33562324,   8.31058537,   1.51562638,   8.41420592,
         2.13062348,  14.28      ,   0.75      ]))
('sequence_3a_564_5_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.620625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.28    ]))
('sequence_3a_564_5_fv1c.npy', ' init cond2', array([ 27.00024053,   8.07779222,   0.62062622,   8.55154416,
         1.33562353,   8.33675713,   1.51562635,   8.05085945,
         2.1306238 ,  14.28      ,   1.        ]))
('sequence_1b_6_7_fv1c.npy', ' init cond2', array([  5.      ,   0.      ,   0.490625,   0.      ,   1.365625,
         0.      ,   1.415625,   0.      ,   2.253125, 

...

('sequence_2c_510_1_fv1c.npy', ' init cond2', array([  5.      ,   0.      ,   0.490625,   0.      ,   1.365625,
         0.      ,   1.415625,   0.      ,   2.253125,   1.028   ,  13.28    ]))
('sequence_2c_510_1_fv1c.npy', ' init cond2', array([  5.00022995,   8.08186014,   0.49062385,   8.08186014,
         1.36562385,   8.08186014,   1.41562615,   8.08186014,
         2.25312385,  13.28      ,   1.25      ]))
('sequence_1d_166_7_fv1c.npy', ' init cond2', array([ 27.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  14.28  ]))
('sequence_1d_166_7_fv1c.npy', ' init cond2', array([ 27.00030011,   8.47764173,   0.72499854,   8.47764173,
         1.47499868,   8.47764173,   1.52500148,   8.47764173,
         2.2374985 ,  14.28      ,   1.75      ]))
('sequence_2c_504_5_fv1c.npy', ' init cond2', array([  5.      ,   0.      ,   0.490625,   0.      ,   1.365625,
         0.      ,   1.415625,   0.      ,   2.253125,   1.028   ,  11.28  

('sequence_1d_276_27_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.410625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  12.28    ]))
('sequence_1d_276_27_fv1c.npy', ' init cond2', array([ 27.00020219,   6.08136161,   0.41062454,   7.33753211,
         1.33562407,   7.01426664,   1.51562579,   7.60764587,
         2.13062399,  12.28      ,   1.428     ]))
('sequence_1d_202_13_fv1c.npy', ' init cond2', array([ 27.    ,   0.    ,   0.725 ,   0.    ,   1.475 ,   0.    ,
         1.525 ,   0.    ,   2.2375,   1.428 ,  14.28  ]))
('sequence_1d_202_13_fv1c.npy', ' init cond2', array([ 27.00032684,   9.18295871,   0.72499815,   8.50229332,
         1.47499856,   8.90061431,   1.52500167,   8.8417176 ,
         2.23749837,  14.28      ,   0.75      ]))
('sequence_1l_434_5_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.410625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.

         2.23749883,  14.28      ,   0.75      ]))
('sequence_1l_437_8_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.410625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.28    ]))
('sequence_1l_437_8_fv1c.npy', ' init cond2', array([ 27.00033108,   8.10976359,   0.41062378,   8.3118876 ,
         1.33562365,   8.71039293,   1.51562657,   8.84105517,
         2.13062334,  14.28      ,   0.5       ]))
('sequence_2b_454_5_fv1c.npy', ' init cond2', array([ 19.      ,   0.      ,   0.420625,   0.      ,   1.345625,
         0.      ,   1.425625,   0.      ,   2.215625,   1.428   ,  14.28    ]))
('sequence_2b_454_5_fv1c.npy', ' init cond2', array([ 19.00027191,   8.47764173,   0.42062364,   8.47764173,
         1.34562356,   8.47764173,   1.42562633,   8.47764173,
         2.21562354,  14.28      ,   1.25      ]))
('sequence_1k_327_8_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.410625,   0.      ,   1.335625,
  

('sequence_1c_94_5_fv1c.npy', ' init cond2', array([  5.      ,   0.      ,   0.490625,   0.      ,   1.365625,
         0.      ,   1.415625,   0.      ,   2.253125,   1.428   ,  14.28    ]))
('sequence_1c_94_5_fv1c.npy', ' init cond2', array([  5.00028211,   8.40381779,   0.49062371,   8.59079127,
         1.36562359,   8.04401932,   1.41562628,   8.7938229 ,
         2.25312355,  14.28      ,   0.75      ]))
('sequence_1b_22_3_fv1c.npy', ' init cond2', array([  5.      ,   0.      ,   0.490625,   0.      ,   1.365625,
         0.      ,   1.415625,   0.      ,   2.253125,   1.428   ,  14.28    ]))
('sequence_1b_22_3_fv1c.npy', ' init cond2', array([  5.00032665,   8.80727082,   0.49062337,   8.51872706,
         1.36562356,   8.36501973,   1.41562634,   8.00537072,
         2.25312374,  14.28      ,   1.75      ]))
('sequence_3a_702_4_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.620625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   

('sequence_1l_400_1_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.410625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.28    ]))
('sequence_1l_400_1_fv1c.npy', ' init cond2', array([ 27.0002237 ,   8.96650686,   0.41062335,   7.77360904,
         1.3356239 ,   9.49535244,   1.51562702,   7.82613599,
         2.13062388,  14.28      ,   0.75      ]))
('sequence_3a_601_3_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.620625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625,   1.428   ,  14.28    ]))
('sequence_3a_601_3_fv1c.npy', ' init cond2', array([ 27.00025704,   9.49727741,   0.62062694,  10.14323447,
         1.33562262,   8.40819403,   1.51562642,   7.99730567,
         2.13062371,  14.28      ,   1.75      ]))
('sequence_1l_360_1_fv1c.npy', ' init cond2', array([ 27.      ,   0.      ,   0.410625,   0.      ,   1.335625,
         0.      ,   1.515625,   0.      ,   2.130625

In [3]:
unique_init_conds_test = []
test_seqname_initcond_dict = {}

for files in test_set_filenames:
    data_load_path = test_path + '/data/' + files[0]
    label_load_path = test_path + '/label/' + files[1]
    # print("data/label load path: {} \n {}".format(data_load_path,label_load_path))
    test_array = np.load(data_load_path)
    test_label_array = np.load(label_load_path)[:, 1:]
    if test_array.shape[1] == 12:
        test_array = test_array[:,1:] #skip step_index. 
    if "fv1b" not in str(files[0]):
        print(test_array[0,:])
        #init_cond = train_array[0,[3,5,7,9]]
        init_cond = test_array[0,[2,4,6,8]]
    if "fv1b" in str(files[0]):
        init_cond = test_array[0,[2,4,6,8]]
    print(str(files[0])," init cond",init_cond)
    test_seqname_initcond_dict[str(files[0])] = str(init_cond)
    
    array_equality = False
    if len(unique_init_conds_test) == 0:
            unique_init_conds_test.append(init_cond)
    if len(unique_init_conds_test) != 0:
        for ic in unique_init_conds_test:
            if np.array_equal(init_cond,ic) == True:
                #print("equal array found in list?", np.array_equal(load_seq,load_sequence))
                array_equality = True
    if array_equality == False:
        unique_init_conds_test.append(init_cond) #the load types
    #print("first 6 rows:",train_array[0:2,:])
print("there are ",len(unique_init_conds_test)," init conds in the TESTING set.", unique_init_conds_test)
print("and ", len(unique_init_conds_test), "in the TRAINING set.",unique_init_conds)
#print("unique init conds:",unique_init_conds_test)

#any(item.shape != (24,) for item in flattened_eq_len_loadseq_list)
#print("any non overlapping init conds?", cmp(unique_init_conds,unique_init_conds_test))

[ 19.         0.         0.420625   0.         1.345625   0.         1.425625
   0.         2.215625   1.428     14.28    ]
('sequence_2b_458_9_fv1c.npy', ' init cond', array([ 0.420625,  1.345625,  1.425625,  2.215625]))
[ 27.         0.         0.410625   0.         1.335625   0.         1.515625
   0.         2.130625   1.428     12.28    ]
('sequence_1l_419_10_fv1c.npy', ' init cond', array([ 0.410625,  1.335625,  1.515625,  2.130625]))
[ 19.         0.         0.420625   0.         1.345625   0.         1.425625
   0.         2.215625   1.428     14.28    ]
('sequence_2b_479_10_fv1c.npy', ' init cond', array([ 0.420625,  1.345625,  1.425625,  2.215625]))
[  5.         0.         0.490625   0.         1.365625   0.         1.415625
   0.         2.253125   1.428     12.28    ]
('sequence_1c_118_9_fv1c.npy', ' init cond', array([ 0.490625,  1.365625,  1.415625,  2.253125]))
[ 27.         0.         0.620625   0.         1.335625   0.         1.515625
   0.         2.130625   1.428  

[ 27.         0.         0.410625   0.         1.335625   0.         1.515625
   0.         2.130625   1.428     14.28    ]
('sequence_1l_369_10_fv1c.npy', ' init cond', array([ 0.410625,  1.335625,  1.515625,  2.130625]))
[ 27.         0.         0.410625   0.         1.335625   0.         1.515625
   0.         2.130625   1.428     14.28    ]
('sequence_1l_379_10_fv1c.npy', ' init cond', array([ 0.410625,  1.335625,  1.515625,  2.130625]))
[  5.         0.         0.490625   0.         1.365625   0.         1.415625
   0.         2.253125   1.428     14.28    ]
('sequence_1b_48_9_fv1c.npy', ' init cond', array([ 0.490625,  1.365625,  1.415625,  2.253125]))
[  5.         0.         0.490625   0.         1.365625   0.         1.415625
   0.         2.253125   1.428     14.28    ]
('sequence_1c_108_9_fv1c.npy', ' init cond', array([ 0.490625,  1.365625,  1.415625,  2.253125]))
[ 27.       0.       0.725    0.       1.475    0.       1.525    0.
   2.2375   1.428   14.28  ]
('sequence_1d

[ 27.         0.         0.410625   0.         1.335625   0.         1.515625
   0.         2.130625   1.428     14.28    ]
('sequence_1k_358_9_fv1c.npy', ' init cond', array([ 0.410625,  1.335625,  1.515625,  2.130625]))
[ 27.         0.         0.410625   0.         1.335625   0.         1.515625
   0.         2.130625   1.428     14.28    ]
('sequence_1l_408_9_fv1c.npy', ' init cond', array([ 0.410625,  1.335625,  1.515625,  2.130625]))
[ 27.         0.         0.620625   0.         1.335625   0.         1.515625
   0.         2.130625   1.428     14.28    ]
('sequence_3a_648_10_fv1c.npy', ' init cond', array([ 0.620625,  1.335625,  1.515625,  2.130625]))
[  5.         0.         0.490625   0.         1.365625   0.         1.415625
   0.         2.253125   1.028     15.28    ]
('sequence_2c_529_10_fv1c.npy', ' init cond', array([ 0.490625,  1.365625,  1.415625,  2.253125]))
[  5.         0.         0.490625   0.         1.365625   0.         1.415625
   0.         2.253125   1.428  

In [4]:
seqname_initcond_dict_initcondkey = {}

#combined_seqname_initcond_dict = {}

#do this twice. once for the training and once for the testing set
for key,value in training_seqname_initcond_dict.iteritems():
    print("key",key,"value",value)
    combined_seqname_initcond_dict[key[:-4]]=value
    
    #load_sequence_aslist = str(unique_load_sequences_aslists[value])
    #this arrangement below is necessary because dict has to have a list as its value. 
    if seqname_initcond_dict_initcondkey.get(value) != None:
        seqname_initcond_dict_initcondkey[value].append(key)
    if seqname_initcond_dict_initcondkey.get(value) == None: 
        content_list = []
        content_list.append(key[:-4])
        seqname_initcond_dict_initcondkey[value] = content_list

for key,value in test_seqname_initcond_dict.iteritems():
    print("key",key,"value",value)
    combined_seqname_initcond_dict[key[:-4]]=value
    #load_sequence_aslist = str(unique_load_sequences_aslists[value])
    #this arrangement below is necessary because dict has to have a list as its value. 
    if seqname_initcond_dict_initcondkey.get(value) != None:
        seqname_initcond_dict_initcondkey[value].append(key)
    if seqname_initcond_dict_initcondkey.get(value) == None: 
        content_list = []
        content_list.append(key[:-4])
        seqname_initcond_dict_initcondkey[value] = content_list

        
#print(filename_loadseq_dict_loadlistposkey)
sn_ic_ick_df = pd.DataFrame.from_dict(seqname_initcond_dict_initcondkey,orient='index')
sn_ic_ick_df.index.rename(name='init_cond',inplace=True)

sn_ic_snk_df = pd.DataFrame.from_dict(combined_seqname_initcond_dict,orient='index')
sn_ic_snk_df.index.rename(name='seq_name',inplace=True)
sn_ic_snk_df.columns=['init_cond']

sn_ic_ick_df.to_csv(analysis_path + "seqname_initcond_dict_initcondkey.csv") #the 3 init conds are index
sn_ic_snk_df.to_csv(analysis_path + "combined_seqname_initcond_dict.csv") #seq_name is index. to join later. 


('key', 'sequence_3a_651_13_fv1c.npy', 'value', '[ 0.610625  1.335625  1.515625  2.130625]')
('key', 'sequence_1l_363_4_fv1c.npy', 'value', '[ 0.410625  1.335625  1.515625  2.130625]')
('key', 'sequence_3a_664_6_fv1c.npy', 'value', '[ 0.620625  1.335625  1.515625  2.130625]')
('key', 'sequence_1d_175_16_fv1c.npy', 'value', '[ 0.725   1.475   1.525   2.2375]')
('key', 'sequence_1k_331_2_fv1c.npy', 'value', '[ 0.410625  1.335625  1.515625  2.130625]')
('key', 'sequence_3a_593_15_fv1c.npy', 'value', '[ 0.610625  1.335625  1.515625  2.130625]')
('key', 'sequence_2c_514_5_fv1c.npy', 'value', '[ 0.490625  1.365625  1.415625  2.253125]')
('key', 'sequence_3a_573_14_fv1c.npy', 'value', '[ 0.620625  1.335625  1.515625  2.130625]')
('key', 'sequence_3a_663_5_fv1c.npy', 'value', '[ 0.620625  1.335625  1.515625  2.130625]')
('key', 'sequence_1d_127_8_fv1c.npy', 'value', '[ 0.410625  1.335625  1.515625  2.130625]')
('key', 'sequence_1l_390_1_fv1c.npy', 'value', '[ 0.410625  1.335625  1.515625  2.13

In [5]:
#right df is the one described in the cell above. 
master_per_loadseq_df = pd.read_csv(analysis_path + 'master_score_per_loadseq.csv',index_col=0) #left. seq_name is the index. 
results_seqname_initcond_df = pd.merge(master_per_loadseq_df,sn_ic_snk_df,how='inner',left_index=True,right_index=True)
print(results_seqname_initcond_df.head(2))
print(results_seqname_initcond_df.columns)
print(results_seqname_initcond_df.index)


                          mse_avg   mse_med  mse_stdev   mae_avg   mae_med  \
seq_name                                                                     
sequence_2c_539_10_fv1c  0.085384  0.011081   0.119663  0.136150  0.011572   
sequence_2c_538_9_fv1c   0.085634  0.010419   0.119506  0.136272  0.011706   

                         mae_stdev  mae_normd_dev    mape_avg    mape_med  \
seq_name                                                                    
sequence_2c_539_10_fv1c   0.180962            NaN  125.078974  133.284079   
sequence_2c_538_9_fv1c    0.180844            NaN  141.869085  151.855119   

                         mape_stdev  msle_avg  msle_med  msle_stdev  \
seq_name                                                              
sequence_2c_539_10_fv1c   42.506095  0.048854  0.041356    0.016204   
sequence_2c_538_9_fv1c    43.579381  0.048836  0.041387    0.016188   

                                                         init_cond  
seq_name                

In [6]:
columns_to_cast = list(results_seqname_initcond_df.columns)
columns_to_cast.remove('init_cond')
print(columns_to_cast)

for colname in columns_to_cast: #for some reason it's all objects...gotta cast to numeric. 
    results_seqname_initcond_df[str(colname)] = pd.to_numeric(results_seqname_initcond_df[str(colname)],errors='coerce')
for colname in columns_to_cast:
    print("new!", colname, results_seqname_initcond_df[colname].dtype)
grouped_sn_ic_df = results_seqname_initcond_df.groupby(by=['init_cond']).mean()

print(grouped_sn_ic_df.head(2))
#print(type(grouped_sn_ic_df))

pd.DataFrame(grouped_sn_ic_df).to_csv(analysis_path+'sn_ic_grouped_by_ic.csv')

['mse_avg', 'mse_med', 'mse_stdev', 'mae_avg', 'mae_med', 'mae_stdev', 'mae_normd_dev', 'mape_avg', 'mape_med', 'mape_stdev', 'msle_avg', 'msle_med', 'msle_stdev']
('new!', 'mse_avg', dtype('float64'))
('new!', 'mse_med', dtype('float64'))
('new!', 'mse_stdev', dtype('float64'))
('new!', 'mae_avg', dtype('float64'))
('new!', 'mae_med', dtype('float64'))
('new!', 'mae_stdev', dtype('float64'))
('new!', 'mae_normd_dev', dtype('float64'))
('new!', 'mape_avg', dtype('float64'))
('new!', 'mape_med', dtype('float64'))
('new!', 'mape_stdev', dtype('float64'))
('new!', 'msle_avg', dtype('float64'))
('new!', 'msle_med', dtype('float64'))
('new!', 'msle_stdev', dtype('float64'))
                                            mse_avg   mse_med  mse_stdev  \
init_cond                                                                  
[ 0.410625  1.335625  1.515625  2.130625]  0.124295  0.019757   0.148329   
[ 0.420625  1.345625  1.425625  2.215625]  0.135983  0.013930   0.173708   

                 

In [7]:
grouped_sn_ic_df2 = results_seqname_initcond_df.groupby(by=['init_cond']).count()
pd.DataFrame(grouped_sn_ic_df2).to_csv(analysis_path+'sn_ic_grouped_by_ic_count.csv')